In [ ]:
# chage the follwing link where your file was saved on your google drive

share_="1LOviPZ8AcTR9ViquQuhpqcO65MHdLDmp"
url_head='https://drive.google.com/uc?id='

dwn_url=url_head+share_

In [ ]:
!pip install -q gdown


In [ ]:
import gdown

In [ ]:
file_='traibing_0.csv.zip'
data=file_
gdown.download(dwn_url,data , quiet=False)

In [ ]:
import pandas as pd


In [ ]:
!pip install -q git+https://github.com/cchuang2009/tw_matplotlib.git
import tw_matplotlib

In [ ]:
file= 'traibing_0.csv.zip'
df= pd.read_csv(file, compression='zip')

In [ ]:
target = '飆股'

df_test=pd.read_csv("http://120.126.22.75/tbrain/public_x.csv.zip")

In [ ]:
import gc
gc.collect()

In [ ]:
# contenance df and df_test
chunk=pd.concat([df, df_test], ignore_index=True)
df_size=len(df)
df_test_size=len(df_test)

del df,df_test

In [ ]:
y= chunk[target]
chunk= chunk.drop(columns=[target])


In [ ]:
# delete the object, ID , column
drop_cols = ['ID']
chunk.drop(drop_cols, axis=1, inplace=True)

In [ ]:
print("df size:", df_size)
print("df_test size:", df_test_size)
print("chunk size:", len(chunk))

In [ ]:
def bin_feature_qcut(x, n_bins=100):
    try:
        return pd.qcut(x, q=n_bins, labels=False, duplicates="drop")
    except:
        return pd.Series(np.nan, index=x.index)

In [ ]:
# save the columns to file
col_=list(chunk.columns)
with open('columns.txt', 'w') as f:
    for item in col_:
        f.write("%s\n" % item)


In [ ]:
from collections import defaultdict

# Load updated columns
with open("columns.txt", encoding="utf-8") as f:
    all_columns = [line.strip() for line in f.readlines()]

# Categorize based on keyword hints
categories = defaultdict(list)

for col in all_columns:
    if "RSI" in col:
        categories["RSI"].append(col)
    elif "MACD" in col:
        categories["MACD"].append(col)
    elif "DIF" in col:
        categories["DIF"].append(col)
    elif "ADX" in col or "DI" in col:
        categories["Trend Indicators"].append(col)
    elif "乖離" in col or "偏離" in col:
        categories["BIAS"].append(col)
    elif "營收" in col or "財報" in col or "EPS" in col or "ROE" in col:
        categories["Financial"].append(col)
    elif "主力" in col or "買賣超" in col:
        categories["Chip/Main Force"].append(col)
    elif "券商" in col or "外資" in col or "投信" in col:
        categories["Institution"].append(col)
    elif "成交量" in col or "成交金額" in col:
        categories["Volume"].append(col)
    elif "指標" in col:
        categories["Technical_Other"].append(col)
    else:
        categories["Other"].append(col)

In [ ]:
chunk['個股前1天成交量'].hist(bins=50)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

In [ ]:
# 📦 Category-aware binning using qcut, log, or linear strategies
def bin_features_by_category(df, categories, n_bins=100):
    import numpy as np
    import pandas as pd

    df_binned = df.copy()

    def safe_qcut(x):
        try:
            return pd.qcut(x, q=n_bins, labels=False, duplicates="drop")
        except:
            return pd.Series(np.nan, index=x.index)

    def safe_digitize(x):
        x_clean = x.dropna()
        if x_clean.empty:
            return pd.Series(np.nan, index=x.index)
        col_min, col_max = x_clean.min(), x_clean.max()
        if col_min == col_max:
            return pd.Series(0, index=x.index)
        bins = np.linspace(col_min, col_max, n_bins + 1)
        binned = np.digitize(x, bins[1:-1])
        return pd.Series(binned, index=x.index)

    def log_binning(x):
        x_safe = x.copy()
        x_safe = np.sign(x_safe) * np.log1p(np.abs(x_safe))
        return safe_qcut(x_safe)

    for cat, cols in categories.items():
        for col in cols:
            if col not in df.columns:
                continue
            if cat in ["RSI", "MACD", "DIF", "BIAS", "Trend Indicators", "Chip/Main Force", "Institution"]:
                df_binned[col] = safe_qcut(df[col])
            elif cat in ["Volume"]:
                df_binned[col] = log_binning(df[col])
            elif cat == "Financial":
                df_binned[col] = log_binning(df[col])  # or safe_qcut(df[col]) if values are bounded
            elif cat == "Technical_Other":
                df_binned[col] = safe_qcut(df[col])
            else:  # 'Other' or uncategorized
                df_binned[col] = safe_digitize(df[col])

    return df_binned

In [ ]:
# --- Step 1: Prepare Dataset ---
#df_size = 200864
chunk = bin_features_by_category(chunk, categories, n_bins=100)  # bin all columns using qcut
X_train = chunk.iloc[:df_size].copy()
X_test = chunk.iloc[df_size:].copy()
y_train = y[:df_size].copy()

In [ ]:
# --- Step 2: Train CV RandomForest ---
def train_cv_rf(X, y, X_test):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    oof = np.zeros(len(X))
    preds = np.zeros(len(X_test))
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):

        #X_tr, y_tr = X.iloc[train_idx], y.iloc[train_idx]
        #X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        #sampler = ImbPipeline([
        #    ("over", SMOTE(sampling_strategy=0.1, random_state=42)),
        #    ("under", RandomUnderSampler(sampling_strategy=0.5, random_state=42))
        #])
        #X_res, y_res = sampler.fit_resample(X_tr, y_tr)

        model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)


        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        #model = RandomForestClassifier(n_estimators=100, class_weight=None, random_state=42)
        #model.fit(X_res, y_res)
        prob = model.predict_proba(X.iloc[val_idx])[:, 1]
        oof[val_idx] = prob
        preds += model.predict_proba(X_test)[:, 1] / skf.n_splits
        pred_label = (prob > 0.2).astype(int)
        #prob = model.predict_proba(X_val)[:, 1]
        #oof[val_idx] = prob
        #preds += model.predict_proba(X_test)[:, 1] / skf.n_splits
        #pred_label = (prob > 0.2).astype(int)
        print(f"[RF Fold {fold}] AUC: {roc_auc_score(y.iloc[val_idx], prob):.4f}, F1: {f1_score(y.iloc[val_idx], pred_label):.4f}, Acc: {accuracy_score(y.iloc[val_idx], pred_label):.4f}")
        #print(f"[RF Fold {fold}] AUC: {roc_auc_score(y_val, prob):.4f}, F1: {f1_score(y_val, pred_label):.4f}, Acc: {accuracy_score(y_val, pred_label):.4f}")

    return oof, preds,model

In [ ]:
# --- Step 5: Run All Models ---
oof_rf, pred_rf, model_rf = train_cv_rf(X_train, y_train, X_test)

In [ ]:
model_rf[0]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(20, 10))
plot_tree(model_rf.estimators_[0],
          feature_names=X_train.columns.tolist(),
          filled=True,
          max_depth=3,
          fontsize=8)
plt.title("Example Tree from Random Forest")
plt.show()

In [ ]:
# Assuming model_rf is trained and you have feature names
importances = model_rf.feature_importances_
features = X_train.columns

# Sort and plot
importance_df = pd.DataFrame({"feature": features, "importance": importances})
importance_df = importance_df.sort_values(by="importance", ascending=False).head(30)

plt.figure(figsize=(10, 8))
plt.barh(importance_df["feature"], importance_df["importance"])
plt.title("Top 30 Feature Importances (Random Forest)")
plt.gca().invert_yaxis()
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

In [ ]:
thread=0.2
df_test[target]=(preds_f1 > thread).astype(int)
df_test.to_csv('submission-4-17.csv', index=False)